In [1]:
import fitz
from tqdm import tqdm

import pandas as pd
import numpy as np

In [48]:
def text_format(text):
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

In [49]:
def open_read_pdf(pdf_name):
    doc = fitz.open(pdf_name)
    pages_and_text = []

    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_format(text)
        pages_and_text.append({"page_number": page_num,
                               "page_char_number": len(text.split(" ")),
                               "page_token_number": len(text.split(" "))/4,
                               "page_sentence_number": len(text.split('.')),
                               "page_text": text})

    return pages_and_text 

In [50]:
pages_and_text = open_read_pdf('support_cours_bdd.pdf')
pages_and_text[46]

149it [00:00, 573.38it/s]


{'page_number': 46,
 'page_char_number': 218,
 'page_token_number': 54.5,
 'page_sentence_number': 12,
 'page_text': "Chapitre 2 : Conception des bases de données avec le modèle Entité Association    46    Figure 28 : Association réflexive  NOTE  Dans le cas d’une association réflexive, les rôles doivent être mentionnés clairement sur le diagramme.  Dans une association réflexive il est primordial de spécifier les rôles et les identifier  avec leurs noms. Ceci permet de distinguer les rôles que jouent deux entités du même type- entité dans l'association réflexive.  L’utilisation des associations réflexives induit beaucoup de cas indésirables qu’il  faudra gérer par la suite en utilisant les contraintes. Par exemple, dans le cas présenté ci- dessus, on devrait s’assurer que ne soit pas la même entité qui participe en tant que époux et  épouse à la fois.   4.5. Les Domaines De Valeur  Certaines informations ne peuvent accepter qu’un ensemble déterminé et limité de  valeurs. Elles sont do

In [51]:
df = pd.DataFrame(pages_and_text)
df

,page_number,page_char_number,page_token_number,page_sentence_number,page_text
0,0,36,9.00,5,Fouad DAHAK Ecole Supérieure d’Informatique...
1,1,569,142.25,23,1 Préambule Après onze années en tant que ch...
2,2,52,13.00,2,"2 Pour finir, je pense avoir synthétisé ma mo..."
3,3,251,62.75,3224,3 Table des matières Présentation du cours...
4,4,310,77.50,3332,4 3.6.2. L'Association 1.1 - 0.n ou 1.1 - 1.n...
...,...,...,...,...,...
144,144,227,56.75,64,"144 Bibliographie 1. Abrams, Marshall D., ..."
145,145,277,69.25,1,145 Annexe 1 : Les Fonctions dans SQL 92 Do...
146,146,114,28.50,1,146 IS [NOT] NULL NULL INNER JOIN Jointure...
147,147,233,58.25,20,147 Annexe 2 : Outils Pédagogiques 1. Functi...


In [52]:
df.describe().round(2)

,page_number,page_char_number,page_token_number,page_sentence_number
count,149.00,149.00,149.00,149.00
mean,74.00,261.23,65.31,201.00
std,43.16,100.58,25.14,730.89
min,0.00,36.00,9.00,1.00
25%,37.00,193.00,48.25,10.00
50%,74.00,251.00,62.75,14.00
75%,111.00,325.00,81.25,20.00
max,148.00,569.00,142.25,3635.00


### Splitting page into sentences

In [53]:
from spacy.lang.en import English

nlp = English()
nlp.add_pipe('sentencizer')

In [54]:
def split_into_sentences_spacy(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences, len(sentences)

In [55]:
df['sentences'], df['spacy_sentences_num'] = zip(*df['page_text'].apply(split_into_sentences_spacy))

In [56]:
df.head()

,page_number,page_char_number,page_token_number,page_sentence_number,page_text,sentences,spacy_sentences_num
0,0,36,9.00,5,Fouad DAHAK Ecole Supérieure d’Informatique...,[Fouad DAHAK Ecole Supérieure d’Informatiqu...,1
1,1,569,142.25,23,1 Préambule Après onze années en tant que ch...,[1 Préambule Après onze années en tant que c...,22
2,2,52,13.00,2,"2 Pour finir, je pense avoir synthétisé ma mo...","[2 Pour finir, je pense avoir synthétisé ma m...",2
3,3,251,62.75,3224,3 Table des matières Présentation du cours...,[3 Table des matières Présentation du cour...,29
4,4,310,77.50,3332,4 3.6.2. L'Association 1.1 - 0.n ou 1.1 - 1.n...,"[4 3.6.2., L'Association 1.1 - 0.n ou 1.1 - 1...",36


In [57]:
for i in range(len(df['sentences'][1])):
    print(df['sentences'][1][i])

1  Préambule  Après onze années en tant que chargé de cours du module « Bases De Données » à  l’Ecole Supérieure d’Informatique, je me rappelle toujours du coup de fil de Mr Medjaoui  (Directeur des études de l’époque) me demandant d’assurer ce cours pour les étudiants de  3ème année option « Systèmes d’Information ».
A peine recruté, je me retrouve comme  chargé de cours devant 150 étudiants, tous assoiffés de savoir et qui en demandaient toujours  d’avantage.
 N’ayant pas une grande expérience en pédagogie, ce fut uniquement mon expérience  professionnelle en tant que développeur et administrateur de bases de données qui a poussé  le DE à me désigner pour cette tâche.
J’ai commencé alors à chercher des supports de cours  pour m’imprégner du module et préparer mes leçons, mais, hélas, je n’ai pas trouvé ne serait- ce qu’un brouillon.
Je me suis alors réfugié à la bibliothèque de l’école où j’ai pratiquement  dévoré tous les livres qui parlaient du domaine.
J’en ai lu ainsi une dizaine

### Devide the lists of sentences into chunks

In [58]:
chunk_size = 20
def split_into_chunks(sentences, chunk_size):
    chunks = []
    if len(sentences) <= chunk_size:
        num_chunks = 1
        return sentences, num_chunks
    else:
        for i in range(0,len(sentences), chunk_size):
            chunk = sentences[i:i + chunk_size]
            chunks.append(chunk)
            num_chunks = i
    return chunks, num_chunks

In [61]:
split_into_chunks(df['sentences'][1],10)

([['1  Préambule  Après onze années en tant que chargé de cours du module « Bases De Données » à  l’Ecole Supérieure d’Informatique, je me rappelle toujours du coup de fil de Mr Medjaoui  (Directeur des études de l’époque) me demandant d’assurer ce cours pour les étudiants de  3ème année option « Systèmes d’Information ».',
   'A peine recruté, je me retrouve comme  chargé de cours devant 150 étudiants, tous assoiffés de savoir et qui en demandaient toujours  d’avantage.',
   ' N’ayant pas une grande expérience en pédagogie, ce fut uniquement mon expérience  professionnelle en tant que développeur et administrateur de bases de données qui a poussé  le DE à me désigner pour cette tâche.',
   'J’ai commencé alors à chercher des supports de cours  pour m’imprégner du module et préparer mes leçons, mais, hélas, je n’ai pas trouvé ne serait- ce qu’un brouillon.',
   'Je me suis alors réfugié à la bibliothèque de l’école où j’ai pratiquement  dévoré tous les livres qui parlaient du domaine.'

In [62]:
df['chunks'], df['num_chunks'] = zip(*df['sentences'].apply(lambda x: split_into_chunks(x, chunk_size=10)))
df['chunks'][1]

[['1  Préambule  Après onze années en tant que chargé de cours du module « Bases De Données » à  l’Ecole Supérieure d’Informatique, je me rappelle toujours du coup de fil de Mr Medjaoui  (Directeur des études de l’époque) me demandant d’assurer ce cours pour les étudiants de  3ème année option « Systèmes d’Information ».',
  'A peine recruté, je me retrouve comme  chargé de cours devant 150 étudiants, tous assoiffés de savoir et qui en demandaient toujours  d’avantage.',
  ' N’ayant pas une grande expérience en pédagogie, ce fut uniquement mon expérience  professionnelle en tant que développeur et administrateur de bases de données qui a poussé  le DE à me désigner pour cette tâche.',
  'J’ai commencé alors à chercher des supports de cours  pour m’imprégner du module et préparer mes leçons, mais, hélas, je n’ai pas trouvé ne serait- ce qu’un brouillon.',
  'Je me suis alors réfugié à la bibliothèque de l’école où j’ai pratiquement  dévoré tous les livres qui parlaient du domaine.',
  '

In [32]:
df.describe().round(2)

,page_number,page_char_number,page_token_number,page_sentence_number,spacy_sentences_num,num_chunks
count,149.00,149.00,149.00,149.00,149.00,149.00
mean,74.00,261.23,65.31,201.00,11.72,6.92
std,43.16,100.58,25.14,730.89,7.44,7.14
min,0.00,36.00,9.00,1.00,1.00,1.00
25%,37.00,193.00,48.25,10.00,7.00,1.00
50%,74.00,251.00,62.75,14.00,11.00,10.00
75%,111.00,325.00,81.25,20.00,15.00,10.00
max,148.00,569.00,142.25,3635.00,48.00,40.00


### combine sentences of each chunk to a text paraph

In [69]:
import re

pattern = r'([.,])(?=[^\s])'
sentences_map = {}
pages_and_chunks = []

for i, item in tqdm(enumerate(df['chunks'])):
    for j in range(len(item)):
        chunk_dict = {}
        joined_sentences_chunk = item[j][0] if len(item[j]) == 1 else ''.join(item[j]).strip()
        spaced_paragraphe = re.sub(pattern, r'\1 ', joined_sentences_chunk)
        chunk_dict['page_number'] = i
        chunk_dict['sentence_chunk'] = spaced_paragraphe
        chunk_dict['tokens_number'] = len(spaced_paragraphe.split(' ')) / 4

        pages_and_chunks.append(chunk_dict)

149it [00:00, 5234.27it/s]


In [71]:
paragraphs_df = pd.DataFrame(pages_and_chunks)
paragraphs_df

,page_number,sentence_chunk,tokens_number
0,0,Fouad DAHAK Ecole Supérieure d’Informatique...,10.00
1,1,1 Préambule Après onze années en tant que ch...,67.75
2,1,Deux choses qu’on ne nous apprend pas à l’éco...,56.00
3,1,J’ai pratiquement fait le tour des livres et ...,17.00
4,2,"2 Pour finir, je pense avoir synthétisé ma mo...",10.75
...,...,...,...
609,148,Ceci via une interface intuitive et très facil...,2.75
610,148,La version actuelle offre un ensemble de fonct...,3.75
611,148,Frai est le premier outil dans son genre. . . ...,16.50
612,148,Les cinq agrégats de base sont également implé...,4.00


In [72]:
paragraphs_df.describe().round(2)

,page_number,tokens_number
count,614.00,614.00
mean,84.50,27.02
std,41.47,71.77
min,0.00,0.25
25%,51.00,4.50
50%,89.50,8.00
75%,119.00,17.94
max,148.00,888.00


In [81]:
def process_rows(df):
    for index, row in df.iterrows():
        text = row['sentence_chunk']
        print(text)
        
        decision = input(f"Do you want to delete this row number {index} ? (y/n): ").strip().lower()
        
        if decision == 'y':
            # Delete the row
            df.drop(index, inplace=True)
            print(f"{index} Row deleted")
        elif decision == 'n':
            print(f"{index}Row kept")
        else:
            print("Invalid input. Skipping this row.")
            break
        
        print("=" * 30) 

In [84]:
process_rows(paragraphs_df)

1  Préambule  Après onze années en tant que chargé de cours du module « Bases De Données » à  l’Ecole Supérieure d’Informatique, je me rappelle toujours du coup de fil de Mr Medjaoui  (Directeur des études de l’époque) me demandant d’assurer ce cours pour les étudiants de  3ème année option « Systèmes d’Information ». A peine recruté, je me retrouve comme  chargé de cours devant 150 étudiants, tous assoiffés de savoir et qui en demandaient toujours  d’avantage. N’ayant pas une grande expérience en pédagogie, ce fut uniquement mon expérience  professionnelle en tant que développeur et administrateur de bases de données qui a poussé  le DE à me désigner pour cette tâche. J’ai commencé alors à chercher des supports de cours  pour m’imprégner du module et préparer mes leçons, mais, hélas, je n’ai pas trouvé ne serait- ce qu’un brouillon. Je me suis alors réfugié à la bibliothèque de l’école où j’ai pratiquement  dévoré tous les livres qui parlaient du domaine. J’en ai lu ainsi une dizaine 

In [85]:
paragraphs_df.to_c sv('bdd.csv', index=False) 